# pyHDX basics

In [3]:
from pyhdx import PeptideMasterTable, read_dynamx
import os

In [5]:
data_dir = '../tests/test_data'
filename = 'ecSecB_apo.csv'
fpath = os.path.join(data_dir, filename)

AttributeError: module 'pyhdx' has no attribute 'models'

We can use the ``read_dynamx`` function to read the file. This function returns a ``numpy`` structured array where each
entry corresponds to one peptide, in this example 567 peptides. 

In [ ]:
data = read_dynamx(fpath)
len(data)

 This array is loaded into the ``PeptideMasterTable`` class, which is the main data entry class. By specifying ``drop_first`` 
 the number of n-terminal residues to remove can be changed and with ``ignore_prolines`` prolines residues, which do not
 have exchanging amide hydrogens, can be ignored.


In [ ]:
master_table = PeptideMasterTable(data, drop_first=1, ignore_prolines=True)

This master table allows us to control how the deuterium uptake content is determined. The method ``set_control`` can be
used to choose which set of peptides is used as the fully deuterated (FD) control. This adds a new field called 'uptake'
which is the normalized (to 100%) deuterium uptake of each peptide. 

In [ ]:
master_table.set_control(('Full deuteration control', 0.167))
master_table.data['uptake'][:50]

Next we'll split the data and group them by the different states. This returns a dictionary where the values are
all peptides for a given state. The peptides for each state are grouped by their exposure time, forming a ``KineticSeries`` 
object

In [ ]:
states = master_table.groupby_state()
print(states.keys())

In [6]:
series = states['SecB WT apo']
type(series), len(series), series.timepoints

dict_keys(['Full deuteration control', 'SecB WT apo'])


Iterating over a ``KineticSeries`` object returns a set of ``PeptideMeasurements`` each with their own attributes describing
the topology of the coverage. When all ``PeptideMeasurements`` in the series have identical coverage, the series is said
to be ``uniform``, which can be checked by the ``uniform`` property. Series can be made ``uniform`` by default, removing peptides which are not found in all timepoints. ``KineticsSeries`` are required to be 
uniform before fitting them.  

In [ ]:
print(series.uniform)
series.make_uniform()  # This series already is uniform